In [ ]:
# import the necessary packages

from imutils.video import FileVideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import math
import time
import dlib
import cv2
import numpy as np
import sys
import subprocess as sp
from sklearn.decomposition import PCA
from PIL import Image
import csv
from os import listdir
from scipy.spatial.distance import pdist, squareform

w = []

In [ ]:
# load landmark detector

print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
fa = face_utils.FaceAligner(predictor, desiredFaceWidth=256)

In [ ]:
# function to calculate weighted median per pixel

def weighted_median(data, midpoint, weights=None):
    import numpy as np
    if weights is None:
        return np.median(np.array(data).flatten())
    data, weights = np.array(data).flatten(), np.array(weights).flatten()
    if any(weights > 0):
        sorted_data, sorted_weights = map(np.array, zip(*sorted(zip(data, weights))))
        if any(weights > midpoint):
            return (data[weights == np.max(weights)])[0]
        cumulative_weight = np.cumsum(sorted_weights)
        below_midpoint_index = np.where(cumulative_weight <= midpoint)[0][-1]
        if cumulative_weight[below_midpoint_index] - midpoint < sys.float_info.epsilon:
            return np.mean(sorted_data[below_midpoint_index:below_midpoint_index+2])
    return sorted_data[below_midpoint_index+1]

In [ ]:
# function to calculate sigma

def calcSigma(S_n, S_N, S_target, n, N):
    left = 1
    right = 1000
    while left < right:
        mid = int(left + (right - left) / 2)
        start = np.sum(calcWeight(S_n, S_target, n, left/10)) - 0.9*np.sum(calcWeight(S_N, S_target, N, left/10))
        end = np.sum(calcWeight(S_n, S_target, n, right/10)) - 0.9*np.sum(calcWeight(S_N, S_target, N, right/10))
        if start > end:
            right = mid
        else:
            left = mid+1
    return left

In [ ]:
# function to calculate weight, given sigma (to be changed to include sigma)

def calcWeight(S, S_target, len, sigma):
    const = 2*(sigma**2)
    global w 
    w = []
    for i in range(len):
        z = (np.linalg.norm(S[i] - S_target))**2
        w = np.append(w, np.exp(-z/const))
    return w

In [ ]:
# function to mask the target image

def maskFrames(imgs, centres, N):
    new_imgs = []
    centres = centres.astype(int)
    for i in range(N):
        img = cv2.imread(imgs[i])
        v = Image.fromarray(img[(centres[i][1]-50):(centres[i][1]+50), (centres[i][0]-50):(centres[i][0]+50)])
        b, g, r = v.split()
        inter_img = Image.merge("RGB", (r, g, b))
        val = "/home/shuvam/Downloads/facial-landmarks/internal/img"+str(i)+".jpg"
        inter_img.save(val)
        new_imgs = np.append(new_imgs, val)
    return new_imgs

In [ ]:
# function to calculate weighted median for all target images

def weightedMedian(S_all, S_target, target_frame, S_centres, S_centres_n, frames, target_centre, S_vec, S_vec_n, n, N, weights):
    x_num = 100
    y_num = 100
    
    weight_sum = 0.5*np.sum(weights)
    centre_x = target_centre[0] - (y_num/2)
    centre_y = target_centre[1] - (x_num/2)
    masked_images_n = maskFrames(S_vec_n, S_centres_n, n)
    im_res = np.zeros((x_num, y_num, 3))
    S_target_new = cv2.imread(target_frame)
    S_int = np.zeros((100, 100, 3))
    
    b = []
    g = []
    r = []
    
    t = 0
    
    b1 = []
    g1 = []
    r1 = []
    
    for x in range(n):
        img = Image.open(masked_images_n[x])
        B, G, R = img.split()
        B = np.asarray(B)
        G = np.asarray(G)
        R = np.asarray(R)
        if t == 0:
            b = np.append(b, np.reshape(B, (np.product(B.shape),)))
            g = np.append(g, np.reshape(G, (np.product(G.shape),)))
            r = np.append(r, np.reshape(R, (np.product(R.shape),)))
            t = 1
        else:
            b = np.vstack((b, np.reshape(B, (np.product(B.shape),))))
            g = np.vstack((g, np.reshape(G, (np.product(G.shape),))))
            r = np.vstack((r, np.reshape(R, (np.product(R.shape),))))
            
    for i in range (x_num*y_num):
        b1 = np.append(b1, weighted_median(b[:,i], weight_sum, weights = w))
        g1 = np.append(g1, weighted_median(g[:,i], weight_sum, weights = w))
        r1 = np.append(r1, weighted_median(r[:,i], weight_sum, weights = w))
    
    t = 0
    for i in range(x_num):
        for j in range(y_num):
            z = (r1[t], g1[t], b1[t])
            S_target_new[int(centre_y+i)][int(centre_x+j)] = z
            t += 1
            
    return S_target_new

In [ ]:
# calculate pca

def find_PCA(video_link, folder):
    vs = FileVideoStream(video_link).start()
    time.sleep(2.0)
    start_time = time.time()
    
    frameNo = 1
    t = 0
    imgs_rslt = []
    centres = []
    frame_list = []
    length = int(cv2.VideoCapture(video_link).get(cv2.CAP_PROP_FRAME_COUNT))

    # loop over the frames from the video stream
    while True:
        
        frame = vs.read()
        frame = imutils.resize(frame, width=600)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        
        # loop over the face detections
        for rect in rects:
            
            faceAligned = fa.align(frame, gray, rect)
            dirs = folder + str(frameNo) + ".jpg"
            cv2.imwrite(dirs, faceAligned)
            rectan = dlib.rectangle(0, 0, 256, 256)
            grays = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)

            shape = predictor(grays, rectan)
            shape = face_utils.shape_to_np(shape)
            
            shapes = np.concatenate([shape[48:61], shape[62:65]])
            shapes = np.concatenate([shapes, shape[66:69]])
            imgs = np.reshape(shapes, (np.product(shapes.shape),))
            
            if t == 0:
                centres = np.append(centres, np.sum(shapes, axis=0))
                imgs_rslt = np.append(imgs_rslt,imgs)
                t = 1

            else:
                centres = np.vstack((centres, np.sum(shapes, axis=0)))
                imgs_rslt = np.vstack((imgs_rslt,imgs))
            
            frame_list = np.append(frame_list, dirs)

        # increment frame count
        if frameNo < length:
            frameNo += 1
        else:
            vs.stop()
            break

    imgs_rslt = imgs_rslt.astype(int)
    centres = (centres/[18, 18]).astype(int)
    
    pca = PCA(n_components=20)
    img_pca = pca.fit_transform(imgs_rslt)
    
    elapsed_time = time.time() - start_time
    print("Done here...")
    print("Elapsed time - " + str(elapsed_time))
    
    return imgs_rslt, img_pca, frame_list, length, centres

In [ ]:
# calculate pca
# modified image loading

def find_PCA(folder):
    time.sleep(2.0)
    start_time = time.time()
    
    frameNo = 1
    t = 0
    imgs_rslt = []
    centres = []
    frame_list = []
    imagesList = sorted(listdir(folder))
    length = len(imagesList)

    # loop over the frames from the video stream
    for image in imagesList:
        
        frame = cv2.imread(folder + image)
        frame = imutils.resize(frame, width=600)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        
        # loop over the face detections
        for rect in rects:
            
            faceAligned = fa.align(frame, gray, rect)
            dirs = folder + image
            rectan = dlib.rectangle(0, 0, 256, 256)
            grays = cv2.cvtColor(faceAligned, cv2.COLOR_BGR2GRAY)

            shape = predictor(grays, rectan)
            shape = face_utils.shape_to_np(shape)
            
            shapes = np.concatenate([shape[48:61], shape[62:65]])
            shapes = np.concatenate([shapes, shape[66:69]])
            imgs = np.reshape(shapes, (np.product(shapes.shape),))
            
            if t == 0:
                centres = np.append(centres, np.sum(shapes, axis=0))
                imgs_rslt = np.append(imgs_rslt,imgs)
                t = 1

            else:
                centres = np.vstack((centres, np.sum(shapes, axis=0)))
                imgs_rslt = np.vstack((imgs_rslt,imgs))
            
            frame_list = np.append(frame_list, dirs)

        # increment frame count
        frameNo += 1

    imgs_rslt = imgs_rslt.astype(int)
    centres = (centres/[18, 18]).astype(int)
    
    pca = PCA(n_components=20)
    img_pca = pca.fit_transform(imgs_rslt)
    
    elapsed_time = time.time() - start_time
    print("Done here...")
    print("Elapsed time - " + str(elapsed_time))
    
    return imgs_rslt, img_pca, frame_list, length, centres

In [ ]:
# calculate l2 similarity

def similarity(src_pca, tgt_pca, len):
    dist = []
    
    for i in range(len-1):
        if i == 0:
            dist = np.append(dist, [i+1, np.linalg.norm(src_pca[i] - tgt_pca)])
        else:
            dist = np.vstack((dist, [i+1, np.linalg.norm(src_pca[i] - tgt_pca)]))

    dist = dist[dist[:,1].argsort()]
    return dist

In [10]:
# preprocessing

source_vec, source_pca, source_frame, source_len, source_centre = find_PCA("/home/shuvam/Downloads/facial-landmarks/source/")
target_vec, target_pca, target_frame, target_len, target_centre = find_PCA("/home/shuvam/Downloads/facial-landmarks/target/")

Done here...
Done here...


In [ ]:
n = 40
target_len = len(target_vec)
source_len = len(source_vec)
for i in range(94,4600):
    start_time = time.time()
    y = []
    centres = []
    rslt = similarity(target_vec, source_vec[i], target_len)
    frames = []
    rslt1 = rslt
    rslt = rslt[:n]
    t = 0
    
    for z in rslt[:,0].astype(int):
        if t == 0:
            y = np.append(y, target_vec[z-1])
            centres = np.append(centres, target_centre[z-1])
            frames = np.append(frames, target_frame[z-1])
            t = 1
        else:
            y = np.vstack((y, target_vec[z-1]))
            centres = np.vstack((centres, target_centre[z-1]))
            frames = np.append(frames, target_frame[z-1])
            
    y = y.astype(int)
    print("Working on frame " + str(i))
    sigma = calcSigma(y, target_vec, source_vec[i], n, target_len)
    weight = calcWeight(y, target_vec[i], n, sigma)
    gen_img = weightedMedian(source_vec, target_vec[i], target_frame[i], source_centre, centres, frames, target_centre[i], source_frame, frames, n, target_len, weight)
    gen_img = Image.fromarray(gen_img)
    b, g, r = gen_img.split()
    gen_img = Image.merge("RGB", (r, g, b))
    gen_img.save("/home/shuvam/Downloads/facial-landmarks/inter/img"+('0'*(len(str(target_len))-len(str(i))))+str(i)+".jpg")
    elapsed_time = time.time() - start_time
    print("Done")
    print("Elapsed time - " + str(elapsed_time)+'\n')

Working on frame 94
done
Working on frame 95
done
Working on frame 96
done
Working on frame 97
done
Working on frame 98
done
Working on frame 99
done
Working on frame 100
done
Working on frame 101
done
Working on frame 102
done
Working on frame 103
done
Working on frame 104
done
Working on frame 105
done
Working on frame 106
done
Working on frame 107
done
Working on frame 108
done
Working on frame 109
done
Working on frame 110
done
Working on frame 111
done
Working on frame 112
done
Working on frame 113
done
Working on frame 114
done
Working on frame 115
done
Working on frame 116
done
Working on frame 117
done
Working on frame 118
done
Working on frame 119
done
Working on frame 120
done
Working on frame 121
done
Working on frame 122
done
Working on frame 123
done
Working on frame 124
done
Working on frame 125
done
Working on frame 126
done
Working on frame 127
done
Working on frame 128
done
Working on frame 129
done
Working on frame 130
done
Working on frame 131
done
Working on frame 1

In [112]:
# test cell 1 (don't run)

n = 50
target_len = len(target_vec)
y = []
rslt = similarity(target_vec, source_vec[108], target_len)
rslt = rslt[:n]
print(rslt)
t = 0
for z in rslt[:,0].astype(int):
    if t == 0:
        y = np.append(y, target_vec[z-1])
        t = 1
    else:
        y = np.vstack((y, target_vec[z-1]))
y = y.astype(int)
sigma = calcSigma(y, target_vec, source_vec[107], n, target_len)
print(sigma)

[[ 4473.            10.95445115]
 [ 3727.            13.07669683]
 [ 4409.            13.22875656]
 [ 4408.            14.03566885]
 [ 3720.            15.03329638]
 [ 3700.            15.06651917]
 [ 3726.            15.13274595]
 [ 3823.            15.13274595]
 [ 3830.            15.39480432]
 [ 3725.            15.49193338]
 [ 2703.            15.71623365]
 [ 4472.            15.8113883 ]
 [ 3701.            15.87450787]
 [ 3721.            16.0623784 ]
 [ 3931.            16.2788206 ]
 [ 3744.            16.2788206 ]
 [ 2701.            16.55294536]
 [ 3719.            16.64331698]
 [ 3724.            16.70329309]
 [ 2704.            16.73320053]
 [ 2700.            16.73320053]
 [ 3557.            16.76305461]
 [ 3738.            16.79285562]
 [ 4410.            16.82260384]
 [ 2702.            16.85229955]
 [ 3745.            16.91153453]
 [ 4412.            17.        ]
 [ 3829.            17.11724277]
 [ 4414.            17.20465053]
 [  415.            17.29161647]
 [ 3558.  

In [110]:
# test cell 2 (don't run)

n = 50
target_len = len(target_vec)
y = []
rslt = similarity(target_vec, source_vec[108], target_len)
rslt = rslt[:n]
print(rslt)
t = 0
for z in rslt[:,0].astype(int):
    if t == 0:
        y = np.append(y, target_vec[z-1])
        t = 1
    else:
        y = np.vstack((y, target_vec[z-1]))
y = y.astype(int)
sigma = calcSigma(y, target_vec, source_vec[107], n, target_len)
print(sigma)

[[ 4473.            10.95445115]
 [ 3727.            13.07669683]
 [ 4409.            13.22875656]
 [ 4408.            14.03566885]
 [ 3720.            15.03329638]
 [ 3700.            15.06651917]
 [ 3726.            15.13274595]
 [ 3823.            15.13274595]
 [ 3830.            15.39480432]
 [ 3725.            15.49193338]
 [ 2703.            15.71623365]
 [ 4472.            15.8113883 ]
 [ 3701.            15.87450787]
 [ 3721.            16.0623784 ]
 [ 3931.            16.2788206 ]
 [ 3744.            16.2788206 ]
 [ 2701.            16.55294536]
 [ 3719.            16.64331698]
 [ 3724.            16.70329309]
 [ 2704.            16.73320053]
 [ 2700.            16.73320053]
 [ 3557.            16.76305461]
 [ 3738.            16.79285562]
 [ 4410.            16.82260384]
 [ 2702.            16.85229955]
 [ 3745.            16.91153453]
 [ 4412.            17.        ]
 [ 3829.            17.11724277]
 [ 4414.            17.20465053]
 [  415.            17.29161647]
 [ 3558.  

KeyboardInterrupt: 

In [15]:
# TODO
# stitch images into video

img_names = "img%0"+(str(target_len)+".jpg"
FFMPEG_BIN = "ffmpeg"
command = [ FFMPEG_BIN,
        '-y', # (optional) overwrite output file if it exists
        '-f', 'rawvideo',
        '-vcodec','rawvideo',
        '-s', '256x256', # size of one frame
        '-pix_fmt', 'rgb24',
        '-r', '30', # frames per second
        '-i', img_names, # The imput comes from a pipe
        '-i', '/home/shuvam/Downloads/facial-landmarks/target/cut2.mp4',
        '-map', '1:a:0',
        '-shortest',
        '-vcodec', 'mpeg'",
        'result.mp4' ]

pipe = sp.Popen( command, stdin=sp.PIPE, stderr=sp.PIPE)

SyntaxError: invalid syntax (<ipython-input-15-5f6e61e9be66>, line 2)